In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from pathlib import Path
import os
from dotenv import load_dotenv

# 1. 환경변수 로드 (.env 파일에 OPENAI_API_KEY=your_api_key 형태로 저장)
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("❌ OpenAI API 키가 환경변수에 없습니다. .env 파일을 확인하세요.")

# 2. 현재 작업 디렉터리 기준 경로 설정
base_dir = Path(os.getcwd())

# 3. PDF 경로 (상대경로)
pdf_paths = [base_dir / "2023_핵심_컴퓨터활용능력1급_필기.pdf"]

# 4. PDF 파일 존재 여부 체크
for path in pdf_paths:
    if not path.exists():
        raise FileNotFoundError(f"PDF 파일이 없습니다: {path}")

# 5. PDF 문서 로드
all_docs = []
for path in pdf_paths:
    loader = PyPDFLoader(str(path))
    docs = loader.load()
    all_docs.extend(docs)

# 6. PDF 문서를 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
split_docs = text_splitter.split_documents(all_docs)

# 7. 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# 8. 벡터스토어 생성 (FAISS 인덱스 생성)
vectorstore = FAISS.from_documents(split_docs, embeddings)

# 9. 벡터스토어 저장 폴더 생성
save_dir = base_dir / "faiss_db"
save_dir.mkdir(parents=True, exist_ok=True)

# 10. 벡터스토어 로컬 저장
vectorstore.save_local(folder_path=str(save_dir), index_name="index")
print(f"✅ 벡터스토어 저장 완료: {save_dir.resolve()}")

# -----------------------
# 여기서부터 검색 및 QA 체인 생성 코드
# -----------------------

# 11. 저장된 벡터스토어 불러오기 (보안 옵션 True로 설정)
vectorstore = FAISS.load_local(
    str(save_dir),
    embeddings,
    index_name="index",
    allow_dangerous_deserialization=True
)

# 12. 검색기 생성 (유사도 검색, k=3)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 13. 프롬프트 템플릿 생성
prompt_template = """
당신은 친절하고 유능한 전문가입니다. 다음은 사용자의 질문과 관련된 참고 문서입니다:

{context}

질문: {question}

참고 문서를 기반으로 질문에 정확하게 답하세요. 모르는 내용은 추측하지 마세요.
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# 14. 언어 모델 초기화
llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=openai_api_key)

# 15. RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

# 16. 테스트 질의
query = "엑셀에서 함수 IF의 기본적인 사용법은 무엇인가요?"
response = qa_chain.run(query)

print("🔍 질문:", query)
print("💬 답변:", response)
